In [104]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random


def count_duplicates(tensor):
    # Check the shape of the tensor
    if tensor.dim() != 2:
        raise ValueError("Input tensor must be 2-dimensional (length x bits).")

    # Convert the tensor to a set of tuples to find unique rows
    unique_rows = set(map(tuple, tensor.tolist()))
    
    # Calculate the number of duplicates
    num_duplicates = tensor.size(0) - len(unique_rows)
    
    return num_duplicates



def generate_sine_tensor(num_bits, length):
    # Create an array of integers from 0 to length - 1
    t = np.arange(length)
    # Generate the sine waves for each bit
    sine_tensor = np.zeros((length, num_bits))  # Initialize the tensor
    
    for i in range(num_bits):
        frequency = (np.pi / (2 ** i))  # Calculate frequency based on the number of bits
        sine_tensor[:, i] = np.sin(frequency * (t+0.5))  # Fill the tensor with sine values

    return sine_tensor








In [122]:


def rand_repeats(length,repeat_max = 1000,abs_val_max = 1000):
    output = []
    while len(output) < length:
        val = (random.random()-0.5)*2*abs_val_max
        repeats = random.randint(0,1000)
        output.extend([val]*repeats)
    output = output[:length]
    return output

In [114]:


class OnlyPhiLayer(nn.Module):
    def __init__(self, num_bits):
        super(OnlyPhiLayer, self).__init__()
        self.num_bits = num_bits

        self.denominators = torch.tensor([2 ** i for i in range(num_bits)])


    def forward(self, t, phi):
        t_phi_b = torch.full((self.num_bits,), t + phi)
        
        # Compute the sine values using the sine tensor and the modified time
        sine_values = torch.sin(torch.pi/self.denominators * t_phi_b)
        return sine_values


In [123]:
p = OnlyPhiLayer(15)

In [124]:
data_len = 50000
bits = 15


num_bits = bits # Change this for different number of bits
length = data_len   # Change this for different lengths
result_tensor = generate_sine_tensor(num_bits, length)
result_tensor = torch.tensor(result_tensor)

In [125]:
count_duplicates(result_tensor)

0

In [126]:
off_set = 41
test_in = result_tensor[:-off_set].float()
test_out = result_tensor[off_set:].float()
print(test_in.shape, test_out.shape)

torch.Size([49959, 15]) torch.Size([49959, 15])


In [136]:
t = rand_repeats(test_in.shape[0])
test_out = torch.tensor(t).float()
print(test_out.shape)

torch.Size([49959])


In [137]:


class CustomDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

# Example input tensors
# Assuming test_in and test_out are your input and output tensors
# Make sure they are both tensors and have the same length
# test_in = torch.randn(num_samples, input_dim)  # Example input
# test_out = torch.randn(num_samples, output_dim)  # Example output

# Create the dataset
dataset = CustomDataset(test_in, test_out)

# Create a DataLoader
batch_size = 512  # Adjust as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [150]:






# Define a model with a variable number of layers
class MultiLayerModel(nn.Module):
    def __init__(self, input_dim, num_layers):
        super(MultiLayerModel, self).__init__()
        self.layers = nn.ModuleList()

        # Create the specified number of layers
        for _ in range(num_layers):
            self.layers.append(nn.Linear(input_dim, input_dim))  # Each layer has the same input and output dimension

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)  # Pass through the layer
            if i < len(self.layers) - 1:  # Apply ReLU activation only for all but the last layer
                x = F.relu(x)
        return x  # Apply sigmoid activation at the end


class MultiLayerModelOne(nn.Module):
    def __init__(self, input_dim, num_layers):
        super(MultiLayerModelOne, self).__init__()
        self.layers = nn.ModuleList()

        # Create the specified number of layers
        for _ in range(num_layers - 1):  # Create num_layers - 1 linear layers
            self.layers.append(nn.Linear(input_dim, input_dim))  # Each layer has the same input and output dimension
        
        # Add the last layer that reduces output to a single scalar
        self.output_layer = nn.Linear(input_dim, 1)

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)  # Pass through the layer
            if i < len(self.layers) - 1:  # Apply ReLU activation only for all but the last layer
                x = F.relu(x)
        
        x = self.output_layer(x)  # Pass through the output layer to get a scalar
        return x # Return the scalar output




# Example usage
input_dim = test_in.shape[1]
num_layers = 5  # Specify the number of layers

model = MultiLayerModelOne(input_dim, num_layers)

model

MultiLayerModelOne(
  (layers): ModuleList(
    (0-3): 4 x Linear(in_features=15, out_features=15, bias=True)
  )
  (output_layer): Linear(in_features=15, out_features=1, bias=True)
)

In [151]:
# Criterion and optimizer
criterion = nn.MSELoss()  # Change if needed
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 5_000  # Adjust as needed
losses = []  # To store loss values for later analysis

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    for batch_in, batch_out in dataloader:
        optimizer.zero_grad()  # Clear gradients
        
        # Ensure inputs are float
        outputs = model(batch_in.float()).squeeze()  # Forward pass

        # Calculate loss
        loss = criterion(outputs, batch_out.float())
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        losses.append(loss.item())  # Store loss
        
    if (epoch + 1) % 10 == 0:  # Print every 100 epochs
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, evaluate your model on validation data if needed
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    val_outputs = model(validation_data.float())  # Example for validation
    # Evaluate performance on validation data here

# Optionally, plot the losses
import matplotlib.pyplot as plt

plt.plot(range(len(losses)), losses)
plt.xlabel('Batch Iteration')
plt.ylabel('Loss')
plt.title('Training Loss over Batches')
plt.show()


Epoch [10/5000], Loss: 252471.3750
Epoch [20/5000], Loss: 226669.0938
Epoch [30/5000], Loss: 201862.0000
Epoch [40/5000], Loss: 183860.5000
Epoch [50/5000], Loss: 140292.7656
Epoch [60/5000], Loss: 132293.4219
Epoch [70/5000], Loss: 145445.2812
Epoch [80/5000], Loss: 125662.1562
Epoch [90/5000], Loss: 129546.3828
Epoch [100/5000], Loss: 146501.9219
Epoch [110/5000], Loss: 148215.2344
Epoch [120/5000], Loss: 140836.7812
Epoch [130/5000], Loss: 129336.6953
Epoch [140/5000], Loss: 122501.0156
Epoch [150/5000], Loss: 129747.8516
Epoch [160/5000], Loss: 107608.0312
Epoch [170/5000], Loss: 128872.5391
Epoch [180/5000], Loss: 119287.3594
Epoch [190/5000], Loss: 121061.3672
Epoch [200/5000], Loss: 119936.4062
Epoch [210/5000], Loss: 108073.5469
Epoch [220/5000], Loss: 102026.4141
Epoch [230/5000], Loss: 115573.1406
Epoch [240/5000], Loss: 92241.6172
Epoch [250/5000], Loss: 117227.4844
Epoch [260/5000], Loss: 103949.3594
Epoch [270/5000], Loss: 99920.9766
Epoch [280/5000], Loss: 116201.3984
Epo

KeyboardInterrupt: 